In [133]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [134]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [135]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [136]:
pip install qiskit

Note: you may need to restart the kernel to use updated packages.


In [137]:
pip install qiskit[machine-learning]

Note: you may need to restart the kernel to use updated packages.


In [138]:
pip install qiskit-aer-gpu

Note: you may need to restart the kernel to use updated packages.


# Distributed Encoding Protocol for Faster Simulation

## Introduction

### Distributed Amplitude Encoding (DE)
There are two approaches to Quantum Computing, namely Quantum-inspired and Quantum Mechanical, where the latter is known as "true" quantum computing. This paradigm was explored in the passage above under QPIXL encoding protocol, which delved into a new manner of implementing FRQI with a linear scalability. As it can be observed, even with a linear scalability, which allows us to have a significantly lower depth compared to RFV and other Amplitude Encoding protocols, we are still unable to encode large images efficiently without use of a preprocessing step, which was the Autoencoder. Hence in this passage, we will look at a new approach which though may not be reliazable with actual hardware, has promise for a much faster simulation. 

The Distributed Encoder, as the name suggests, relies on a "Divide and Conquer" approach, where the image vector (image after being flattened), can be split into chunks of size n, and we can encode each chunk separately using RFV, hence we can encode images as large as 3,840 x 2,160 pixels in a matter of seconds, with log2(n) qubits, and a shallow depth. The process is as follows :

1) Import dataset

2) Pass image to encoder

3) Split image to smaller chunks and normalize each chunk

4) Encode each chunk using RawFeatureVector (RFV) PQC

5) Simulate each chunk to extract the statevector

6) Append all chunks classically together and renormalize

7) Denormalize the image statevector using RGB_decoder
![image](https://user-images.githubusercontent.com/73689800/221870046-06d47c06-2dc3-4a13-90f4-d47d721d1f8a.png)

By using this approach, imagining a 32 x 32 image, instead of needing a RFV PQC with a depth of 2037 and width of 10, we can use 16 RFV PQCs with depth 121 and width of 6, which allows us to have a much faster simulation. This approach shows itself best as we go higher in resolution (Anything larger than 50 x 50), and thus in circuit volume, and we can see this is perhaps the only manner we can perform the simulated encoding.

First, we will import the necessary packages. For the model, we will be using Qiskit, for plotting we will be using Matplotlib.

In [139]:
import warnings 
warnings.filterwarnings("ignore")

import numpy as np

# Importing standard Qiskit libraries
import qiskit
from typing import Dict, List
from qiskit import QuantumCircuit, transpile, Aer, execute
from qiskit.algorithms.optimizers import COBYLA
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector
from qiskit.quantum_info.operators import Operator
from qiskit_aer import StatevectorSimulator

import pickle
from matplotlib import pyplot as plt
import sklearn

from PIL import Image

print("All packages imported successfully!")

All packages imported successfully!


We will load the image and split it into its RGB channels, namely Red, Green, and Blue.

In [140]:
Image_ghost = Image.open('Ghost.jpg')
Image_ghost_data = np.asarray(Image_ghost)
chunk_size = 64

red_channel = []
green_channel = []
blue_channel = []

for i in range(len(Image_ghost_data)):
    for j in range(len(Image_ghost_data[0])):
        red_channel.append(Image_ghost_data[i][j][0])

for i in range(len(Image_ghost_data)):
    for j in range(len(Image_ghost_data[0])):
        green_channel.append(Image_ghost_data[i][j][1])

for i in range(len(Image_ghost_data)):
    for j in range(len(Image_ghost_data[0])):
        blue_channel.append(Image_ghost_data[i][j][2])
        

red_channel = np.array(red_channel).reshape(len(Image_ghost_data), len(Image_ghost_data[0]))
green_channel = np.array(green_channel).reshape(len(Image_ghost_data), len(Image_ghost_data[0]))
blue_channel = np.array(blue_channel).reshape(len(Image_ghost_data), len(Image_ghost_data[0]))

print(Image_ghost.format)
print(Image_ghost.size)
print(Image_ghost.mode)

JPEG
(1920, 1080)
RGB


In [141]:
Image_ghost_data[0][3]

array([22, 16, 16], dtype=uint8)

In [142]:
Image_ghost_data = np.asarray(Image_ghost)
print(Image_ghost_data.shape)

(1080, 1920, 3)


In [143]:
red_channel_data = np.asarray(red_channel)
print(red_channel_data.shape)

(1080, 1920)


In [144]:
blue_channel_data = np.asarray(blue_channel)
print(blue_channel_data.shape)

(1080, 1920)


In [145]:
green_channel_data = np.asarray(green_channel)
print(green_channel_data.shape)

(1080, 1920)


In [146]:
r_channel_data_flat = red_channel_data.flatten()

In [147]:
b_channel_data_flat = blue_channel_data.flatten()

In [148]:
g_channel_data_flat = green_channel_data.flatten()

In [149]:
len(r_channel_data_flat)

2073600

To see how many qubits we will need for the overall image, we will take the log2 of the image vector. This is essentially to see how many qubits we need to cover all pixels.

In [150]:
int(np.ceil(np.log2(r_channel_data_flat.size)))

21

In some cases, the number of qubits for the datapoint can support even higher number of pixels. For instance, Fashion MNIST uses 28 x 28 images, which means each image has 784 pixels. We will need 10 qubits at least to represent the image, but we can represent a maximum of 1024 pixels. However, even if we don't wish to add any additional pixels, we still need to pad the vector with 0s to get a vector of 1024.

In [130]:
2**int(np.ceil(np.log2(r_channel_data_flat.size)))

2097152

Here is our normalize(x) function. It takes in a vector as an input, and returns the 2-norm normalized vector.

In [151]:
def normalize(x):
    """Here is our normalize(x) function. It takes in a vector as an input, and returns the 2-norm normalized vector.

    Args:
        x (np.array): 
            The image vector.

    Returns:
        (numpy.array): The 2-norm normalized vector.
    """
    x = x.flatten() # First we flatten the image in case it is a 2d array
    normalized_vector = x / np.linalg.norm(x) # Then we normalize the vector to 2-norm
    normalized_vector = [*normalized_vector] # We then format to return an np.array

    return normalized_vector # we return the normalized vector

def de_normalize(normalized_vector, x):
    """Here is our denormalize(x) function. It takes in a 2-norm normalized vector as an input, and returns denormalized vector.

    Args:
        x (np.array): 
            The 2-norm normalized vector.

    Returns:
        (numpy.array): The denormalized vector.
    """
    return normalized_vector * np.linalg.norm(x.flatten()) # We return the denormalized vector by applying the inverse of the normalization factor

We can see how these functions work in the cell below.

## Normalization

Here is each channel before and after normalizing.

In [ ]:
print('before normalize:')
display(plt.imshow(red_channel))

In [ ]:
red_normalized_vector = normalize(red_channel)
#print(normalized_vector)
print('after normalize:')
plt.imshow(np.reshape(np.array(red_normalized_vector),red_channel.shape))

In [ ]:
print('before normalize:')
display(plt.imshow(green_channel))

In [ ]:
green_normalized_vector = normalize(green_channel)
#print(normalized_vector)
print('after normalize:')
plt.imshow(np.reshape(np.array(green_normalized_vector),green_channel.shape))

In [ ]:
print('before normalize:')
display(plt.imshow(blue_channel))

In [ ]:
blue_normalized_vector = normalize(blue_channel)
#print(normalized_vector)
print('after normalize:')
plt.imshow(np.reshape(np.array(blue_normalized_vector),blue_channel.shape))

## Distributed Amplitude Encoding and Decoding with RGB

The PQC we will be using to encode the image chunks will be RawFeatureVector. RawFeatureVector is a shallow PQC used for real-valued amplitude encoding. For a vector of size n, we will need a minimum of ceil(log2(n)) qubits, and must pad the vector to make sure it matches 2^(ceil(log2(n)) = n. 

In [178]:
state = np.ones(4)*1/np.sqrt(4)
print(len(state))
n = 2
qc = RawFeatureVector(2**n)
print("This circuit has " + str(n) + " qubits and " + str(qc.num_parameters) + " parameters.")
print("Below, you can see the circuit")
qc = qc.bind_parameters(state)
qc.decompose(reps=20).draw()

4
This circuit has 2 qubits and 4 parameters.
Below, you can see the circuit


┌────────────┐┌───┐┌──────────┐┌───┐
q_0: ─|0>─┤ U(π/2,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├
          ├────────────┤└─┬─┘└──────────┘└─┬─┘
q_1: ─|0>─┤ U(π/2,0,0) ├──■────────────────■──
          └────────────┘

In [176]:
state = np.ones(32)*1/np.sqrt(32)
print(len(state))
n = 5
qc = RawFeatureVector(2**n)
print("This circuit has " + str(n) + " qubits and " + str(qc.num_parameters) + " parameters.")
print("Below, you can see the circuit")
qc = qc.bind_parameters(state)
qc.decompose(reps=20).draw()

32
This circuit has 5 qubits and 32 parameters.
Below, you can see the circuit


┌────────────┐                                            »
q_0: ─|0>─┤ U(π/2,0,0) ├────────────────────────────────────────────»
          ├────────────┤                                            »
q_1: ─|0>─┤ U(π/2,0,0) ├────────────────────────────────────────────»
          ├────────────┤                      ┌───┐┌──────────┐┌───┐»
q_2: ─|0>─┤ U(π/2,0,0) ├──────────────────────┤ X ├┤ U(0,0,0) ├┤ X ├»
          ├────────────┤┌───┐┌──────────┐┌───┐└─┬─┘└──────────┘└─┬─┘»
q_3: ─|0>─┤ U(π/2,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├──■────────────────┼──»
          ├────────────┤└─┬─┘└──────────┘└─┬─┘                   │  »
q_4: ─|0>─┤ U(π/2,0,0) ├──■────────────────■─────────────────────■──»
          └────────────┘                                            »
«                                                                              »
«q_0: ─────────────────────────────────────────────────────────────────────────»
«                                       ┌───┐┌──────────┐┌───┐┌──────────┐┌───┐»
«q_1: ──────────────────────────────────┤ X ├┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├»
«     ┌──────────┐┌───┐┌──────────┐┌───┐└─┬─┘└──────────┘└─┬─┘└──────────┘└─┬─┘»
«q_2: ┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├──■────────────────┼────────────────■──»
«     └──────────┘└─┬─┘└──────────┘└─┬─┘                   │                   »
«q_3: ──────────────■────────────────┼─────────────────────■───────────────────»
«                                    │                                         »
«q_4: ───────────────────────────────■─────────────────────────────────────────»
«                                                                              »
«                                                                         »
«q_0: ────────────────────────────────────────────────────────────────────»
«     ┌──────────┐┌───┐┌──────────┐┌───┐┌──────────┐┌───┐┌──────────┐┌───┐»
«q_1: ┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├»
«     └──────────┘└─┬─┘└──────────┘└─┬─┘└──────────┘└─┬─┘└──────────┘└─┬─┘»
«q_2: ──────────────┼────────────────■────────────────┼────────────────■──»
«                   │                                 │                   »
«q_3: ──────────────┼─────────────────────────────────■───────────────────»
«                   │                                                     »
«q_4: ──────────────■─────────────────────────────────────────────────────»
«                                                                         »
«                      ┌───┐┌──────────┐┌───┐┌──────────┐┌───┐┌──────────┐┌───┐»
«q_0: ─────────────────┤ X ├┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├»
«     ┌──────────┐┌───┐└─┬─┘└──────────┘└─┬─┘└──────────┘└─┬─┘└──────────┘└─┬─┘»
«q_1: ┤ U(0,0,0) ├┤ X ├──■────────────────┼────────────────■────────────────┼──»
«     └──────────┘└─┬─┘                   │                                 │  »
«q_2: ──────────────┼─────────────────────■─────────────────────────────────┼──»
«                   │                                                       │  »
«q_3: ──────────────┼───────────────────────────────────────────────────────■──»
«                   │                                                          »
«q_4: ──────────────■──────────────────────────────────────────────────────────»
«                                                                              »
«     ┌──────────┐┌───┐┌──────────┐┌───┐┌──────────┐┌───┐┌──────────┐┌───┐»
«q_0: ┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├»
«     └──────────┘└─┬─┘└──────────┘└─┬─┘└──────────┘└─┬─┘└──────────┘└─┬─┘»
«q_1: ──────────────■────────────────┼────────────────■────────────────┼──»
«                                    │                                 │  »
«q_2: ───────────────────────────────■─────────────────────────────────┼──»
«                                                                      │  »
«q_3: ─────────────────────────────────────────────────────────────────┼──»
«             

In [177]:
state = np.ones(128)*1/np.sqrt(128)
print(len(state))
n = 7
qc = RawFeatureVector(2**n)
print("This circuit has " + str(n) + " qubits and " + str(qc.num_parameters) + " parameters.")
print("Below, you can see the circuit")
qc = qc.bind_parameters(state)
qc.decompose(reps=20).draw()

128
This circuit has 7 qubits and 128 parameters.
Below, you can see the circuit


┌────────────┐                                            »
q_0: ─|0>─┤ U(π/2,0,0) ├────────────────────────────────────────────»
          ├────────────┤                                            »
q_1: ─|0>─┤ U(π/2,0,0) ├────────────────────────────────────────────»
          ├────────────┤                                            »
q_2: ─|0>─┤ U(π/2,0,0) ├────────────────────────────────────────────»
          ├────────────┤                                            »
q_3: ─|0>─┤ U(π/2,0,0) ├────────────────────────────────────────────»
          ├────────────┤                      ┌───┐┌──────────┐┌───┐»
q_4: ─|0>─┤ U(π/2,0,0) ├──────────────────────┤ X ├┤ U(0,0,0) ├┤ X ├»
          ├────────────┤┌───┐┌──────────┐┌───┐└─┬─┘└──────────┘└─┬─┘»
q_5: ─|0>─┤ U(π/2,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├──■────────────────┼──»
          ├────────────┤└─┬─┘└──────────┘└─┬─┘                   │  »
q_6: ─|0>─┤ U(π/2,0,0) ├──■────────────────■─────────────────────■──»
          └────────────┘                                            »
«                                                                              »
«q_0: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q_1: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q_2: ─────────────────────────────────────────────────────────────────────────»
«                                       ┌───┐┌──────────┐┌───┐┌──────────┐┌───┐»
«q_3: ──────────────────────────────────┤ X ├┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├»
«     ┌──────────┐┌───┐┌──────────┐┌───┐└─┬─┘└──────────┘└─┬─┘└──────────┘└─┬─┘»
«q_4: ┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├──■────────────────┼────────────────■──»
«     └──────────┘└─┬─┘└──────────┘└─┬─┘                   │                   »
«q_5: ──────────────■────────────────┼─────────────────────■───────────────────»
«                                    │                                         »
«q_6: ───────────────────────────────■─────────────────────────────────────────»
«                                                                              »
«                                                                         »
«q_0: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«q_1: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«q_2: ────────────────────────────────────────────────────────────────────»
«     ┌──────────┐┌───┐┌──────────┐┌───┐┌──────────┐┌───┐┌──────────┐┌───┐»
«q_3: ┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├»
«     └──────────┘└─┬─┘└──────────┘└─┬─┘└──────────┘└─┬─┘└──────────┘└─┬─┘»
«q_4: ──────────────┼────────────────■────────────────┼────────────────■──»
«                   │                                 │                   »
«q_5: ──────────────┼─────────────────────────────────■───────────────────»
«                   │                                                     »
«q_6: ──────────────■─────────────────────────────────────────────────────»
«                                                                         »
«                                                                              »
«q_0: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q_1: ─────────────────────────────────────────────────────────────────────────»
«                      ┌───┐┌──────────┐┌───┐┌──────────┐┌───┐┌──────────┐┌───┐»
«q_2: ─────────────────┤ X ├┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├┤ U(0,0,0) ├┤ X ├»
«     ┌──────────┐┌───┐└─┬─┘└──────────┘└─┬─┘└──────────┘└─┬─┘└──────────┘└─┬─┘»
«q_3: ┤ U(0,0,0) ├┤ X ├──■────────────

We can see how the depth scales exponentially as we go higher. Hence, we can see the value in an approach that utilizes a number of shallower PQCs.

This is the simulate(circ) function, which takes a circuit, and extracts the statevector. We can use either the StatevectorSimulator or design a protocol that extracts this from measurements. 

Using the cell below, we can choose the best parameters for our simulator with respect to precision and GPU. For this event, we utilized the NVIDIA A100 GPU for a substantially faster performance and run of the model.

In [153]:
backend = StatevectorSimulator(precision='double')
backend.available_devices()

('CPU', 'GPU')

In [154]:
import qiskit
from qiskit import BasicAer

def simulate(circ: qiskit.QuantumCircuit) -> dict:
    """This is the simulate(circ) function, which takes a circuit, and extracts the statevector.

    Args:
        circ (qiskit.QuantumCircuit): 
            The quantum circuit representing the encoded vector.

    Returns:
        (dict): The simulated statevector as a dict.
    """
    backend = StatevectorSimulator(precision='double', device = "GPU", max_parallel_threads = 5) # First we initialize our bachend
    job = execute(circ, backend,optimization_level=0) # We run the circuit on the simulator
    result = job.result() # We extract the result from the job
    state_vector = result.get_statevector() # We extract the state vector
    
    histogram = dict() # We initialize dictionary
    for i in range(len(state_vector)): # We iterate over the length of the statevector
        population = abs(state_vector[i]) ** 2 # We take the absolute value
        histogram[i] = population # We append to the histogram
    
    return histogram # We return the histogram

We can now implement our encoding protocol. We will be using two functions for the encoder, and one function for the decoder. The Distributed Encoding Protocol is implemented by using encode_shallow which takes a chunk size, and an image, and internally runs encode_qiskit, which takes a chunk and the number of qubits, and returns the encoded PQC, which evidently gets added to a list which is returns by encode_shallow.

Then we have the decoder, which takes a vector and the original image, and returns the decoded representation. 

In [155]:
def encode_chunk(x,num_qubit):
    """This is the encode_qiskit(x, num_qubit) function, which takes a vector and number of qubits, normalizes it, and returns its encoded representation as a PQC.

    Args:
        x (np.array): 
            The image vector.
        num_qubit (int):
            The number of qubits.

    Returns:
        (qc): The PQC from encoding the vector.
    """
    normalized_vector = normalize(x) # We normalize the vector
    qc = RawFeatureVector(2**num_qubit) # We initialize RFV
    return qc.bind_parameters(normalized_vector) # We encode the vector and return the PQC

def encode_distributed(n, image):
    """This is the encode_shallow(n, image) function, which takes an image and the chunk size, and encodes each chunk separately, and returns the list of PQCs.

    Args:
        n (int): 
            The chunk size.
        image (np.array):
            The image vector.

    Returns:
        (qc_list): The list of PQCs from encoding the vector chunks.
    """
    qubit = int(np.ceil(np.log2(n))) # Getting the qubit number
    print("Number of qubits is " + str(qubit))
    image = image.flatten() # Flatenning the 2D image
    print(len(image))
    state_chunks = [image[i * n:(i + 1) * n] for i in range((len(image) + n - 1) // n)]     # Discretizing the image into chunks of size n
    print("Number of pixels in each chunk is " + str(len(state_chunks[0])))
    print(len(state_chunks))
    chunk_avg = []
    for i in range(len(state_chunks)):
        chunk_avg.append(sum(state_chunks[i])/n)
        
    qc_list = [] # List of PQCs
    counter = 0
    for i in range(len(state_chunks)) : # Iterating over the chunks
        if all(v == 0 for v in state_chunks[i]): # If an entire chunk is made of all 0s, then we simply append it, and do not need to encode it
            qc_list.append(state_chunks[i])
        else:
            qc = encode_chunk(state_chunks[i], qubit) # Else, we pass it to encode_qiskit
            qc_list.append(qc) # And append the qc to the list
        if counter == 100000:
            print(i)
            counter = 0
        else:
            counter+=1
    print(len(qc_list))
    return qc_list, chunk_avg # Lastly, we return the list of PQCs representing each chunk, and the chunk avg ratio

def decode(histogram, data):
    """This is the decode(histogram, data) function, which takes a histogram and the original image, and returns the decoded vector.

    Args:
        histogram (dict): 
            The histogram representing the statevector.
        data (np.array):
            The original image vector

    Returns:
        (np.reshape(de_normalize(after_,data)[:784],data_shape)): The decoded image vector
    """
    new_histogram = {} # First we initialize the histogram
    for key in range(len(qc.parameters)): # Iterating over the keys
        if key in histogram: # If a key is existing in the histogram
            new_histogram[key] = histogram[key] # We simply append it (this protocol is for when we can only return non-zero amplitudes)
        else: # Else if a key is not there
            new_histogram[key] = 0 # We append a 0
    #print(new_histogram)
    after_ = np.array(list(new_histogram.values())) # We convert the histogram to a list of its values
    return np.reshape(de_normalize(after_,data)[:784],data_shape) # And we return the denormalized vector

We will also be implementing two functions for evaluating the encoder with respect to depth and fidelity.

In [156]:
def count_gates(circuit: qiskit.QuantumCircuit) -> Dict[int, int]:
    """This is the count_gates(circuit: qiskit.QuantumCircuit) function, which takes a circuit and returns the number of gate operations with each number of qubits.

    Args:
        circuit (qiskit.QuantumCircuit): 
            The quantum circuit representing a vector.

    Returns:
        (Dict): the number of gate operations with each number of qubits.
    """
    return Counter([len(gate[1]) for gate in circuit.data])

def image_mse(image1,image2):
    """This is the image_mse(image1,image2) function, which takes two images and returns the mean squared error between the two.
        Using sklearns mean squared error:
        https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html

    Args:
        image1 (np.array): 
            An image vector.
        image2 (np.array):
            An image vector.
            
    Returns:
        (mean_squared_error(image1, image2)): the mean squared error between the two images.
    """
    return mean_squared_error(image1, image2)

To put it all together, we will be using extract_image_re function, which takes an image, encodes it using the distributed encoding protocol, simulates it, extracts the overall encoded vector representing the image, denormalizes it, and returns the vector and histogram representation.

In [157]:
def extract_image_re(data, n):
    """This is the extract_image_re(data) function, which takes an image and returns the decoded representation.

    Args:
        data (np.array): 
            The image vector.

    Returns:
        (image_re, hist_new): the the vector and histogram representation of the decoded vector.
    """
    histogram = []
    image_re = []
    hist_new = []
    circuit_list, chunk_avg = encode_distributed(n, data)
    print("Encoding done...")
    for i in range(len(circuit_list)):
        if all(v == 0 for v in circuit_list[i]):
            histogram.append(circuit_list[i])
        else:
            histogram.append(simulate(circuit_list[i]))
        if i>=1000 and i%1000==0:
            print(str(i) + " Circuits have been simulated.")
    
    print("Simulation done...")
    
    for i in range(len(histogram)):
        if type(histogram[i]) is dict:
            temp = list(histogram[i].values())
            for j in range(n):
                hist_new.append(chunk_avg[i]*temp[j])
        else:
            for j in range(n):
                hist_new.append(histogram[i][j])
    
    print("Histogram done...")
    
    print(len(hist_new))
    global_avg = sum(data.flatten())/len(data.flatten())
    hist_new = [hist_new[i] * int(global_avg) for i in range(len(hist_new))]
    hist_new = hist_new / np.linalg.norm(hist_new)
    
    print("Histogram normalization done...")
    
    sum_check = 0
    for i in range(len(hist_new)):
        sum_check+=hist_new[i]**2
    print(sum_check)
    
    image_re = de_normalize(hist_new, Image_ghost_data)
    
    print("Histogram denormalization done...")
    
    print(image_re)
    return image_re, hist_new

Using the function below, we can now pass the extracted normalized channels, and create the RGB image.

In [158]:
def RGB_decoder(red_hist_vector, green_hist_vector, blue_hist_vector):
    """This is the RGB_decoder(red_hist_vector, green_hist_vector, blue_hist_vector) function, which takes three channels and returns the RGB image.

    Args:
        red_hist_vector (np.array): 
            The red channel normalized vector.
        green_hist_vector (np.array): 
            The green channel normalized vector.
        blue_hist_vector (np.array): 
            The blue channel normalized vector.
    Returns:
        (PIL RGB): The pillow RGB image.
    """
    red_max = max(red_hist_vector)
    green_max = max(green_hist_vector)
    blue_max = max(blue_hist_vector)

    red_hist = [red_hist_vector[i]*(255/red_max) for i in range(len(red_hist_vector))]
    blue_hist = [blue_hist_vector[i]*(255/blue_max) for i in range(len(blue_hist_vector))]
    green_hist = [green_hist_vector[i]*(255/green_max) for i in range(len(green_hist_vector))]

    red_vector = np.array(red_hist).reshape(len(Image_ghost_data), len(Image_ghost_data[0]))
    green_vector = np.array(green_hist).reshape(len(Image_ghost_data), len(Image_ghost_data[0]))
    blue_vector = np.array(blue_hist).reshape(len(Image_ghost_data), len(Image_ghost_data[0]))

    rgb = np.dstack((red_vector,green_vector,blue_vector))
    print(rgb.shape)
    
    return Image.fromarray(np.array(rgb).astype(np.uint8), 'RGB')

We will perform the protocol for each channel.

In [159]:
from collections import Counter

dataset_red = [red_channel_data]
print(len(dataset_red))

for data in dataset_red:
    red_image_re,red_hist_vector = extract_image_re(data, chunk_size)    

1
Number of qubits is 6
2073600
Number of pixels in each chunk is 64
32400
32400
Encoding done...
1000 Circuits have been simulated.
2000 Circuits have been simulated.
3000 Circuits have been simulated.
4000 Circuits have been simulated.
5000 Circuits have been simulated.
6000 Circuits have been simulated.
7000 Circuits have been simulated.
8000 Circuits have been simulated.
9000 Circuits have been simulated.
10000 Circuits have been simulated.
11000 Circuits have been simulated.
12000 Circuits have been simulated.
13000 Circuits have been simulated.
14000 Circuits have been simulated.
15000 Circuits have been simulated.
16000 Circuits have been simulated.
17000 Circuits have been simulated.
18000 Circuits have been simulated.
19000 Circuits have been simulated.
20000 Circuits have been simulated.
21000 Circuits have been simulated.
22000 Circuits have been simulated.
23000 Circuits have been simulated.
24000 Circuits have been simulated.
25000 Circuits have been simulated.
26000 Circu

In [ ]:
dataset_blue = [blue_channel_data]
print(len(dataset_red))

for data in dataset_blue:
    blue_image_re,blue_hist_vector = extract_image_re(data, chunk_size)    

1
Number of qubits is 6
2073600
Number of pixels in each chunk is 64
32400
32400
Encoding done...
1000 Circuits have been simulated.
2000 Circuits have been simulated.
3000 Circuits have been simulated.
4000 Circuits have been simulated.
5000 Circuits have been simulated.
6000 Circuits have been simulated.
7000 Circuits have been simulated.
8000 Circuits have been simulated.
9000 Circuits have been simulated.
10000 Circuits have been simulated.
11000 Circuits have been simulated.
12000 Circuits have been simulated.
13000 Circuits have been simulated.
14000 Circuits have been simulated.
15000 Circuits have been simulated.
16000 Circuits have been simulated.
17000 Circuits have been simulated.
18000 Circuits have been simulated.
19000 Circuits have been simulated.
20000 Circuits have been simulated.
21000 Circuits have been simulated.
22000 Circuits have been simulated.
23000 Circuits have been simulated.
24000 Circuits have been simulated.
25000 Circuits have been simulated.
26000 Circu

In [ ]:
dataset_green = [green_channel_data]
print(len(dataset_green))

for data in dataset_green:
    green_image_re,green_hist_vector = extract_image_re(data, chunk_size)    

1
Number of qubits is 6
2073600
Number of pixels in each chunk is 64
32400
32400
Encoding done...
1000 Circuits have been simulated.
2000 Circuits have been simulated.
3000 Circuits have been simulated.
4000 Circuits have been simulated.
5000 Circuits have been simulated.
6000 Circuits have been simulated.
7000 Circuits have been simulated.
8000 Circuits have been simulated.
9000 Circuits have been simulated.
10000 Circuits have been simulated.
11000 Circuits have been simulated.
12000 Circuits have been simulated.
13000 Circuits have been simulated.
14000 Circuits have been simulated.
15000 Circuits have been simulated.
16000 Circuits have been simulated.
17000 Circuits have been simulated.
18000 Circuits have been simulated.
19000 Circuits have been simulated.
20000 Circuits have been simulated.
21000 Circuits have been simulated.
22000 Circuits have been simulated.
23000 Circuits have been simulated.
24000 Circuits have been simulated.
25000 Circuits have been simulated.
26000 Circu

We can see the fidelity of the extracted vectors with the original vectors for each channel.

In [165]:
np.dot(red_hist_vector, red_normalized_vector)

0.9722366432897372

In [166]:
np.dot(green_hist_vector, green_normalized_vector)

0.9729221590395775

In [167]:
np.dot(blue_hist_vector, blue_normalized_vector)

0.9727271466940925

Now we can use RGB_decoder to create our RGB image.

In [ ]:
RGB_decoder(red_hist_vector, green_hist_vector, blue_hist_vector)

Below you can see the decoded 4K Ghost in all his glory with RGB colors, where we split the initial image into three Red, Green, and Blue channels, encode each, and decode them by using RGB_Decoder.
![download](https://user-images.githubusercontent.com/73689800/221864333-8793fb2d-9330-4fae-97d4-5210e249813c.png)